In [1]:
from DistanceSensor import DistanceSensor
from Speaker import Speaker
from Button import Button
from LED import LED

from Theremin import Theremin
from Note import Note
import DataProcessing
#import threading
#import queue
import time
from Studio import Studio

In [2]:
#note distance sensor

noteDistanceSensor = DistanceSensor(24,23)
noteDistanceSensor.read()

/home/csci250/CSCI250/Capstone/DistanceSensor.py:17: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(self.GPIO_TRIGGER, GPIO.OUT)


6.493139266967773

In [3]:
#type distance sensor

typeDistanceSensor = DistanceSensor(22,21)
typeDistanceSensor.read()

75.12496709823608

In [4]:
#make new note
note = Note(noteDistanceSensor.read())

In [5]:
#set up speakers, not forgetting to run sudo pigpiod -p 8887 in terminal

leftSpeaker = Speaker(18)
#leftSpeaker.sing(note)
leftSpeaker.stop()

In [6]:
rightSpeaker = Speaker(19)
#rightSpeaker.sing(note)
rightSpeaker.stop()

In [7]:
#set up button

button = Button(16)

#for i in range(10):
 #   print(button.read())
  #  time.sleep(0.5)

In [8]:
#set up LED
light = LED(27)
light.on()
time.sleep(1)
light.off()

/home/csci250/CSCI250/Capstone/LED.py:9: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(self.ledPin, GPIO.OUT)


In [9]:
#set up Theremin
theremin = Theremin(noteDistanceSensor, typeDistanceSensor, leftSpeaker, rightSpeaker)

In [10]:
#set up the Studio
Studio = Studio(theremin)
Studio.playTheremin(True)
time.sleep(5)
Studio.playTheremin(False)

NameError: name 'dataQ' is not defined

In [ ]:
## Below I write beginning code for the recording studio

In [ ]:
# the theremin reads notes from the sensors and plays them at the same time it is having some problems but I've made some progress

In [ ]:
def dataProducer(dataQ, theremin):
    #This producer collects data from the distance sensor and adds note frequencies to the queue
    while playing == True:
        newNote = theremin.readNote()
            
        
        dataQ.put(newNote)

In [ ]:
def dataConsumer(dataQ, theremin):
    #this consumer gets note frequencies from the queue and plays them on the buzzer
    while playing == True:
        toPlay = dataQ.get()
        theremin.playNote(toPlay)
        

In [ ]:
playing = True

dataQ = queue.Queue()
dataCollection = threading.Thread(target = dataProducer, args = (dataQ,theremin))
dataReading = threading.Thread(target = dataConsumer, args = (dataQ,theremin))

dataCollection.start()
dataReading.start()

dataQ.join()

t0 = time.time()
while time.time() <t0+10:
    playing = True
playing = False

In [24]:
leftSpeaker.stop()
rightSpeaker.stop()

In [ ]:
pi.hardware_PWM(buzPin, 0, 0)

In [ ]:
#below this is a backup plan just in case we need it

t0 = time.time()
data = []
dataFrequencies = []
while time.time() <t0+6:
    newDist = dist1.read()
    if newDist != 0:
        data.append(newDist)
        
    #add newest frequency to frequency array
    for j in range(len(distanceDiscretizations)):
        try:
            if data[-1] > distanceDiscretizations[j] and data[-1] < distanceDiscretizations[j+1]:
                dataFrequencies.append( notes[noteNames[j]] )
                #play this note
                #pi.hardware_PWM(buzPin, dataFrequencies[-1], int(0.5*1e6))
                time.sleep(0.05)
                pi.hardware_PWM(buzPin, 0, 0)
        except:
                dataFrequencies.append( 'Gs' )
                #play this note
                #pi.hardware_PWM(buzPin, dataFrequencies[-1], int(0.5*1e6))
                time.sleep(0.05)
                pi.hardware_PWM(buzPin, 0, 0)
    
data = np.array(data)

In [ ]:
plt.plot(data)

In [ ]:
plt.plot(proc.medianFilter(data))
filteredData = proc.medianFilter(data)

In [ ]:
notes, frequencies = proc.discretizeNotes(filteredData)
plt.plot(frequencies)


In [ ]:
buzPin = 12

pi = pigpio.pi(port = 8887)

for i in range(len(frequencies)):
    pi.hardware_PWM(buzPin, frequencies[i], int(0.5*1e6))
    time.sleep(0.05)
    pi.hardware_PWM(buzPin, 0, 0)